In [203]:
import torch

from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import fpsample

In [204]:
mesh_path = "data/bunny_curvs.ply"
mesh = trimesh.load(mesh_path)
vertices = np.array(mesh.vertices)
sampled_indices = fpsample.fps_sampling(vertices, 200, start_idx=0) # the start index is for reproducibility
print(sampled_indices)

[     0  46368   7436  69677  77678  20057  71103   2730  90036   5350
   8864  41086  49709  19551  40434  91487  34667  10050  38565   9411
  63468  16704  20490  80519  81576  63855  58124  48504  92706  51796
  94371  38128  67960  53545   2501  87985  13974 100591  27348  19775
  92222  15179  94009  32364   6487  36950  84664  73281  41608   7793
  39094   8653  43588  73072  47477  29049  12696  82094  39807  23653
  26506  52606  85780  95785  16527  90621  25345  68974  26272  81156
  57206  74767   4669  62822  29884  72653  99577  60714  93578  96953
  98719  59822   9775  91786  38402  97252  53458  76601  52339  44283
  39255  55355  80176  86884  94588  98258  20251 104168 105316   2885
  50300  72419  67875  54695   5852  62660    160  46469  60294  84359
  36178   6288 103582  25578  71660 101508  18692  59109  19356  47171
  75051  71377  45011   9492  20982  96639  38088  33418  21336  30260
 104618  84753  13874  27010  45613  50695  88070  98646  92580  24228
  7211

In [205]:
grid_range = 1.0
ball_radius = 0.09
lr = 1.0


signatures_full_sampling = torch.zeros((len(sampled_indices), 6), dtype=torch.float32)
signatures_down_sampled = torch.zeros((len(sampled_indices), 6), dtype=torch.float32)
for i,sampled_index in enumerate(sampled_indices):
    dataset_full_sample = create_dataset_from_mesh(mesh_path=mesh_path, sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=1.0)
    print("dataset full size:" , len(dataset_full_sample["train_input"]))
    model_full_sampled = KAN(width=[2, 5, 1], grid=1, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range, grid_range], noise_scale=0.1, noise_scale_base=0.1, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
    model_full_sampled.train(dataset_full_sample, opt="LBFGS", steps=20, lr=lr, lamb=10.0, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
    model_full_sampled = model_full_sampled.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0, 1], [0]])
    model_full_sampled.train(dataset_full_sample, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
     # manual mode
    # if nan continue

    # fix all
    try:
        model_full_sampled.fix_symbolic(0, 0, 0, 'x^4')
        model_full_sampled.fix_symbolic(0, 1, 0, 'x^3')
        model_full_sampled.fix_symbolic(1, 0, 0, 'x^2')
    except:
        print("nan encountered")
        continue


    model_full_sampled.train(dataset_full_sample, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, lamb_coef=0.10, lamb_coefdiff=0.10, loss_fn=gaussian_weighted_mse)

    print(model_full_sampled.symbolic_formula()[0][0])
    x = torch.tensor([[0.0], [0.0]], requires_grad=True).T

    signature_full = calculate_signature(model_full_sampled, x) # only calc sig in the origin


    # down sampling

    dataset_down_sampled = create_dataset_from_mesh(mesh_path=mesh_path, sampled_index=sampled_index, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=0.8)
    print("dataset down sampled size:" , len(dataset_down_sampled["train_input"]))
    model_down_sampled = KAN(width=[2, 5, 1], grid=1, k=5, grid_eps=1.0, seed=0, grid_range=[-grid_range, grid_range], noise_scale=0.1, noise_scale_base=0.1, base_fun=torch.nn.SiLU(), learn_rotation_mat=False)
    model_down_sampled.train(dataset_down_sampled, opt="LBFGS", steps=20, lr=lr, lamb=100.0, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
    model_down_sampled = model_full_sampled.prune(threshold=3e-3, mode='manual', active_neurons_id=[[0, 1], [0]])
    model_down_sampled.train(dataset_down_sampled, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, loss_fn=gaussian_weighted_mse)
     # manual mode

    # fix all
    try:
        model_down_sampled.fix_symbolic(0, 0, 0, 'x^4')
        model_down_sampled.fix_symbolic(0, 1, 0, 'x^3')
        model_down_sampled.fix_symbolic(1, 0, 0, 'x^2')
    except:
        print("nan encountered")
        continue

    model_down_sampled.train(dataset_down_sampled, opt="LBFGS", steps=50, lr=lr, sglr_avoid=True, lamb_coef=0.10, lamb_coefdiff=0.10, loss_fn=gaussian_weighted_mse)

    print(model_down_sampled.symbolic_formula()[0][0])
    x = torch.tensor([[0.0], [0.0]], requires_grad=True).T

    signature_down_sampled = calculate_signature(model_down_sampled, x) # only calc sig in the origin
    # check if signature_full or signature_down_sample is nan
    if torch.isnan(signature_full).any(dim=1) or torch.isnan(signature_down_sampled).any(dim=1):
        print("nan encountered")
        continue

    print("sampled_index", i, "signature ", signature_full)
    signatures_full_sampling[i,:] = signature_full
    print("sampled_index", i, "signature ", signature_down_sampled)
    signatures_down_sampled[i,:] = signature_down_sampled



Number of vertices: 106712
dataset full size: 269


train loss: 3.23e-02 | test loss: 4.95e-02 | reg: 3.90e+00 : 100%|██| 20/20 [00:02<00:00,  8.85it/s]
train loss: 3.81e-03 | test loss: 1.88e+00 | reg: 1.20e+01 : 100%|██| 50/50 [00:01<00:00, 25.19it/s]


r2 is 0.9999845623970032
Best value at boundary.
r2 is 0.999687671661377
r2 is 0.1332828402519226
r2 is not very high, please double check if you are choosing the correct symbolic function.


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:03<00:00, 14.78it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



nan
Number of vertices: 106712
dataset down sampled size: 216


...

9944.34*(-x_2 - 0.03)**6
Number of vertices: 106712
dataset down sampled size: 232


train loss: 3.39e-02 | test loss: 3.36e-02 | reg: 2.43e+00 : 100%|██| 20/20 [00:02<00:00,  9.08it/s]
train loss: 1.00e-03 | test loss: 3.16e-03 | reg: 3.42e+00 : 100%|██| 50/50 [00:01<00:00, 46.84it/s]


r2 is 1.0000004768371582
r2 is 0.9999987483024597
r2 is 0.9999993443489075


train loss: 1.00e-03 | test loss: 3.17e-03 | reg: 4.52e+00 : 100%|██| 50/50 [00:00<00:00, 96.50it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:344: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



7679.63*(-x_2 - 0.03)**6
sampled_index 199 signature  tensor([[-1.6576e-01,  8.1827e-03, -1.2954e+01, -6.0469e+00,  2.0300e+00,
          8.6867e-03]], grad_fn=<StackBackward0>)
sampled_index 199 signature  tensor([[-1.1530e-01, -1.9872e-02,  1.3635e+01,  9.1496e+00, -2.0436e+00,
          1.0615e-02]], grad_fn=<StackBackward0>)


In [206]:
indices_full_thrown = torch.isnan(signatures_full_sampling).any(dim=1)
indices_full_thrown =torch.concatenate([indices_full_thrown, signatures_full_sampling.norm(dim=1) == 0])
indices_down_sampled_thrown = torch.isnan(signatures_down_sampled).any(dim=1)
indices_down_sampled_thrown = torch.concatenate([indices_down_sampled_thrown, signatures_down_sampled.norm(dim=1) == 0])

# throw 0's
signatures_full_sampling = signatures_full_sampling[signatures_full_sampling.norm(dim=1) != 0]
signatures_down_sampled = signatures_down_sampled[signatures_down_sampled.norm(dim=1) != 0]
# throw nan's
signatures_full_sampling = signatures_full_sampling[~torch.isnan(signatures_full_sampling).any(dim=1)]
signatures_down_sampled = signatures_down_sampled[~torch.isnan(signatures_down_sampled).any(dim=1)]


In [208]:
# indice_thrown indices, not logical
# indices_full_thrown = torch.concatenate(indices_full_thrown)
print(indices_full_thrown.shape)
print(torch.arange(len(indices_full_thrown))[indices_full_thrown])

print(indices_down_sampled_thrown.shape)
print(torch.arange(len(indices_down_sampled_thrown))[indices_down_sampled_thrown])

torch.Size([400])
tensor([200, 201, 205, 208, 211, 217, 231, 233, 241, 251, 287, 289, 300, 306,
        311, 323, 325, 337, 346, 348, 372])
torch.Size([400])
tensor([200, 201, 205, 208, 211, 217, 231, 233, 241, 251, 287, 289, 300, 306,
        311, 323, 325, 337, 346, 348, 372])


In [209]:
# normalize each entry to 1 in each of the 6 columns of the signature
signatures_full_sampling = signatures_full_sampling / torch.norm(signatures_full_sampling, dim=0)
signatures_down_sampled = signatures_down_sampled / torch.norm(signatures_down_sampled, dim=0)
print(signatures_full_sampling)
print(signatures_down_sampled)

tensor([[ 1.1755e-04,  1.0764e-06,  2.6139e-04,  1.0356e-05,  2.1062e-05,
          2.0811e-07],
        [-1.7858e-04, -2.0987e-06,  1.9456e-03, -1.5406e-06,  2.8783e-06,
          3.0782e-06],
        [ 2.2794e-02, -6.3087e-06,  1.1474e-05, -2.3884e-03,  1.8209e-04,
          1.7922e-06],
        ...,
        [ 5.8616e-02, -5.3815e-02, -3.4121e-02,  8.1355e-04, -1.6093e-02,
          3.4285e-02],
        [ 5.1302e-04,  6.8678e-05,  3.0752e-04, -4.3398e-06,  1.0395e-04,
         -1.3169e-06],
        [-3.9270e-04,  5.9088e-05, -4.0416e-03, -1.2231e-05,  2.6115e-04,
          1.1491e-07]], grad_fn=<DivBackward0>)
tensor([[ 1.1336e-04,  5.9826e-07,  1.9809e-04,  1.1306e-05,  1.9860e-05,
          1.8226e-07],
        [-2.0963e-04, -2.7446e-06,  2.0566e-03, -1.7555e-06,  4.6756e-06,
          5.3779e-06],
        [-1.2233e-03,  4.8712e-04, -6.3483e-03, -2.5446e-06,  4.9162e-04,
          4.3218e-05],
        ...,
        [ 1.5102e-02, -7.1598e-02,  7.7076e-04,  5.9758e-04,  3.3740e-03,
  

In [210]:
# measure correspondence according distance between signatures
# before normalized each entry to 1

distances = torch.cdist(signatures_full_sampling, signatures_down_sampled, p=2)
print(distances)


tensor([[6.3457e-05, 1.8249e-03, 6.7783e-03,  ..., 7.4096e-02, 4.3498e-04,
         3.8177e-03],
        [1.7718e-03, 1.1530e-04, 8.3881e-03,  ..., 7.4162e-02, 1.9097e-03,
         2.1270e-03],
        [2.2808e-02, 2.3218e-02, 2.4966e-02,  ..., 7.3018e-02, 2.2384e-02,
         2.3551e-02],
        ...,
        [9.4510e-02, 9.5394e-02, 9.3536e-02,  ..., 7.6698e-02, 9.4276e-02,
         9.6076e-02],
        [4.2855e-04, 1.8964e-03, 6.9023e-03,  ..., 7.4078e-02, 1.3655e-04,
         3.8455e-03],
        [4.2770e-03, 6.1066e-03, 2.4998e-03,  ..., 7.4400e-02, 4.3242e-03,
         8.1060e-03]], grad_fn=<ViewBackward0>)


In [211]:
# take minimum distance for each row and assign index which is the closest for each surface
min_distances, min_indices = torch.min(distances, dim=1)
print(min_distances)
print(min_indices)

tensor([6.3457e-05, 1.1530e-04, 3.6033e-03, 5.5524e-04, 4.5107e-05, 1.4835e-05,
        3.8035e-04, 1.2857e-02, 1.8700e-03, 6.1028e-03, 7.8105e-04, 4.1310e-04,
        1.0477e-04, 2.5404e-05, 1.7712e-03, 1.5912e-01, 4.7744e-04, 3.8239e-04,
        2.3299e-03, 1.4496e-04, 2.0317e-05, 3.3686e-04, 8.5701e-05, 8.7977e-05,
        2.8448e-05, 8.2484e-03, 3.6074e-03, 1.1012e-03, 3.0135e-04, 2.0140e-04,
        2.8841e-04, 1.3178e-02, 1.6601e-04, 4.0618e-05, 2.7433e-04, 1.4052e-04,
        8.4702e-04, 6.4269e-03, 2.7920e-05, 3.1635e-05, 6.0539e-03, 1.0723e-03,
        1.5347e-03, 2.0156e-04, 2.9879e-03, 8.9253e-03, 8.3176e-04, 8.3668e-04,
        4.7255e-05, 9.9645e-06, 1.4403e-05, 9.5039e-04, 9.8012e-04, 2.4767e-04,
        1.7408e-03, 6.6683e-06, 4.4593e-04, 1.9642e-04, 6.7023e-04, 3.2152e-05,
        4.4899e-04, 3.5495e-05, 3.5440e-03, 2.9396e-04, 8.7377e-04, 4.0629e-04,
        7.3239e-04, 1.3945e-04, 1.3112e-03, 1.3995e-04, 1.8588e-04, 3.5249e-04,
        1.0879e-04, 7.2865e-03, 1.9561e-

In [212]:
signatures_full_sampling

tensor([[ 1.1755e-04,  1.0764e-06,  2.6139e-04,  1.0356e-05,  2.1062e-05,
          2.0811e-07],
        [-1.7858e-04, -2.0987e-06,  1.9456e-03, -1.5406e-06,  2.8783e-06,
          3.0782e-06],
        [ 2.2794e-02, -6.3087e-06,  1.1474e-05, -2.3884e-03,  1.8209e-04,
          1.7922e-06],
        ...,
        [ 5.8616e-02, -5.3815e-02, -3.4121e-02,  8.1355e-04, -1.6093e-02,
          3.4285e-02],
        [ 5.1302e-04,  6.8678e-05,  3.0752e-04, -4.3398e-06,  1.0395e-04,
         -1.3169e-06],
        [-3.9270e-04,  5.9088e-05, -4.0416e-03, -1.2231e-05,  2.6115e-04,
          1.1491e-07]], grad_fn=<DivBackward0>)

In [213]:
signature_down_sampled

tensor([[-1.1530e-01, -1.9872e-02,  1.3635e+01,  9.1496e+00, -2.0436e+00,
          1.0615e-02]], grad_fn=<StackBackward0>)

In [214]:
torch.norm(signatures_full_sampling[0] - signatures_down_sampled[1])

tensor(0.0018, grad_fn=<LinalgVectorNormBackward0>)

In [215]:
# sum all the min_indices whice equal their index, for example if 0 is in 0 index, add 1 to the counter
torch.sum(min_indices == torch.arange(len(min_indices)))

tensor(162)

In [216]:
# test of distances only using the first entry of the signature, then the first and second, then the first, second and third and so on til the fifth
for i in range(6):
    distances = torch.cdist(signatures_full_sampling[:, :i+1], signatures_down_sampled[:, :i+1], p=2)
    min_distances, min_indices = torch.min(distances, dim=1)
    print("i", i, "min_distances", min_distances, "min_indices", min_indices)
    print("i", i, "sum", torch.sum(min_indices == torch.arange(len(min_indices))))

i 0 min_distances tensor([4.1864e-06, 7.6287e-06, 1.0544e-04, 2.0510e-06, 1.3914e-05, 1.1615e-05,
        1.6859e-07, 1.7860e-04, 1.5554e-05, 1.2581e-04, 1.4674e-05, 1.5079e-06,
        4.9497e-06, 1.5451e-06, 1.7438e-04, 1.2259e-04, 1.1596e-04, 1.1129e-04,
        1.5582e-03, 7.3380e-05, 1.8201e-05, 1.5355e-05, 1.0126e-05, 2.2175e-05,
        1.0126e-05, 2.4561e-04, 2.5590e-05, 1.0343e-05, 9.9237e-05, 8.6753e-06,
        9.3229e-05, 2.8962e-04, 1.9192e-05, 1.2006e-05, 6.0886e-05, 7.4463e-05,
        9.2154e-06, 6.6322e-04, 1.2206e-05, 6.5727e-06, 1.8916e-04, 2.4202e-05,
        9.7820e-05, 2.7139e-05, 1.2028e-03, 1.2234e-03, 1.8532e-04, 1.1364e-05,
        9.2690e-06, 2.5302e-06, 1.0241e-05, 4.4985e-06, 4.3800e-06, 6.9716e-05,
        2.4613e-05, 8.4294e-08, 0.0000e+00, 7.2501e-05, 8.8376e-06, 5.1426e-06,
        2.5536e-04, 5.8157e-06, 4.0766e-05, 1.1680e-06, 6.7435e-07, 3.0864e-05,
        2.6716e-05, 7.1344e-05, 7.0525e-07, 2.4558e-05, 1.4893e-05, 2.9301e-05,
        5.2109e-06, 2.

In [218]:
# i want to check per entry which one is the closest and then do a major vote
# for example surface i first entry is closest to surface j and second is closest to k and so on then i new_sig = [j,k,..,..,...] in the end we majority vote which closest, if j appear the most in i new sig then i is closest to j
for i in range(6):
    distances = torch.cdist(signatures_full_sampling[:, i:i+1], signatures_down_sampled[:, i:i+1], p=2)
    min_distances, min_indices = torch.min(distances, dim=1)
    print("i", i, "min_distances", min_distances, "min_indices", min_indices)
    print("i", i, "sum", torch.sum(min_indices == torch.arange(len(min_indices))))


i 0 min_distances tensor([4.1864e-06, 7.6287e-06, 1.0544e-04, 2.0510e-06, 1.3914e-05, 1.1615e-05,
        1.6859e-07, 1.7860e-04, 1.5554e-05, 1.2581e-04, 1.4674e-05, 1.5079e-06,
        4.9497e-06, 1.5451e-06, 1.7438e-04, 1.2259e-04, 1.1596e-04, 1.1129e-04,
        1.5582e-03, 7.3380e-05, 1.8201e-05, 1.5355e-05, 1.0126e-05, 2.2175e-05,
        1.0126e-05, 2.4561e-04, 2.5590e-05, 1.0343e-05, 9.9237e-05, 8.6753e-06,
        9.3229e-05, 2.8962e-04, 1.9192e-05, 1.2006e-05, 6.0886e-05, 7.4463e-05,
        9.2154e-06, 6.6322e-04, 1.2206e-05, 6.5727e-06, 1.8916e-04, 2.4202e-05,
        9.7820e-05, 2.7139e-05, 1.2028e-03, 1.2234e-03, 1.8532e-04, 1.1364e-05,
        9.2690e-06, 2.5302e-06, 1.0241e-05, 4.4985e-06, 4.3800e-06, 6.9716e-05,
        2.4613e-05, 8.4294e-08, 0.0000e+00, 7.2501e-05, 8.8376e-06, 5.1426e-06,
        2.5536e-04, 5.8157e-06, 4.0766e-05, 1.1680e-06, 6.7435e-07, 3.0864e-05,
        2.6716e-05, 7.1344e-05, 7.0525e-07, 2.4558e-05, 1.4893e-05, 2.9301e-05,
        5.2109e-06, 2.

In [217]:
## check correspondence using cosine similarity
cosine_similarity = torch.nn.CosineSimilarity(dim=1)
print(signatures_full_sampling.shape)
print(signatures_down_sampled.shape)
# cosine_sim = cosine_similarity(signatures_full_sampling, signatures_down_sampled)
# print(cosine_sim)
# correspondence = torch.argmax(cosine_sim, dim=1)
# print(correspondence)

torch.Size([179, 6])
torch.Size([179, 6])


179